In [1]:
import pandas as pd
from cloudant.client import Cloudant
from cloudant.result import Result
from Cloudant_cred import serviceUsername, servicePassword, serviceURL
import datetime
import numpy as np

In [2]:
client = Cloudant(serviceUsername, servicePassword, url=serviceURL)
client.connect()

In [3]:
session = client.session()
# print('Username: {0}'.format(session['userCtx']['name']))
print('Databases: {0}'.format(client.all_dbs()))

Databases: ['ballscrew', 'noderedflvfz20210419', 'training']


In [4]:
traindb = client['ballscrew']

In [5]:
result_collection = Result(traindb.all_docs, include_docs=True)

In [6]:
Data = pd.DataFrame.from_records(pd.DataFrame.from_records(result_collection.all())['doc']).drop(['_id','_rev'], axis=1)
# Data

In [7]:
client.disconnect()

In [8]:
Data['Datetime'] = pd.DataFrame((datetime.datetime.fromtimestamp(d) for d in Data['timestamp']/1000))
# Data

In [9]:
Data.sort_values('timestamp', inplace=True)
Data.drop('timestamp', axis=1, inplace=True)
# Data

In [10]:
Data.reset_index(inplace=True)
Data.drop('index', axis=1, inplace=True)
# Data

In [11]:
import tensorflow as tf
# import matplotlib.pyplot as plt

In [12]:
Data = Data[Data.Step < 5]
Data = Data[Data.Step > 0]

In [13]:
Abnormal = Data[Data['class'] == 1]
Normal = Data[Data['class'] == 0][Data.AX != -1][Data.AX != 0]

<ipython-input-13-1650cf57bb52>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  Normal = Data[Data['class'] == 0][Data.AX != -1][Data.AX != 0]
<ipython-input-13-1650cf57bb52>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  Normal = Data[Data['class'] == 0][Data.AX != -1][Data.AX != 0]


In [14]:
# fig, ax = plt.subplots(1, 2, figsize=(14, 14))
# ax[0].scatter(Normal['Step'], Normal['Current'], alpha=0.1)
# ax[1].scatter(Abnormal['Step'], Abnormal['Current'], alpha=0.1)
#
# ax[0].set_ylim((-3e6, 3e6))
# ax[1].set_ylim((-3e6, 3e6))

In [15]:
# fig, ax = plt.subplots(4,1, figsize=(14,28))
# ax[0].plot(Normal.Datetime, Normal.Current)
# ax[1].plot(Normal.Datetime, Normal.AX)
# ax[2].plot(Normal.Datetime, Normal.AY)
# ax[3].plot(Normal.Datetime, Normal.AZ)

In [16]:
# condition = (Normal.Datetime > datetime.datetime(2021, 6, 11, 20, 41)) & (Normal.Datetime < datetime.datetime(2021, 6, 11, 20, 41, 20))
condition1 = (Normal.Datetime > datetime.datetime(2021, 6, 11, 16, 46, 45)) & \
             (Normal.Datetime < datetime.datetime(2021, 6, 11, 20, 41, 10))
condition2 = (Normal.Datetime < datetime.datetime(2021, 6, 10, 19, 54)) & \
             (Normal.Datetime < datetime.datetime(2021, 6, 16))
condition3 = (Normal.Datetime > datetime.datetime(2021, 7, 4)) & \
             (Normal.Datetime < datetime.datetime(2021, 7, 5, 5, 35, 48))
condition4 = (Normal.Datetime > datetime.datetime(2021, 7, 5, 15, 7, 0)) & \
             (Normal.Datetime < datetime.datetime(2021, 7, 5, 17, 6, 17))
condition5 = (Normal.Datetime > datetime.datetime(2021, 7, 5, 20, 30, 15)) & \
            (Normal.Datetime < datetime.datetime(2021, 7, 5, 22))

condition = condition1 | condition2 | condition3 | condition4 | condition5

Normal_filtered = Normal[condition]
Abnormal_fault = Normal[~condition]
Abnormal_fault['class'] = 1

# fig, ax = plt.subplots(4,1, figsize=(14,28))
# ax[0].plot(Normal_filtered.Datetime, Normal_filtered.Current)
# ax[1].plot(Normal_filtered.Datetime, Normal_filtered.AX)
# ax[2].plot(Normal_filtered.Datetime, Normal_filtered.AY)
# ax[3].plot(Normal_filtered.Datetime, Normal_filtered.AZ)

<ipython-input-16-677eeba95760>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Abnormal_fault['class'] = 1


In [17]:
# fig, ax = plt.subplots(4,1, figsize=(14,28))
# ax[0].plot(Abnormal_fault.Datetime, Abnormal_fault.Current)
# ax[1].plot(Abnormal_fault.Datetime, Abnormal_fault.AX)
# ax[2].plot(Abnormal_fault.Datetime, Abnormal_fault.AY)
# ax[3].plot(Abnormal_fault.Datetime, Abnormal_fault.AZ)
# Abnormal_fault

In [18]:
# fig, ax = plt.subplots(4,1, figsize=(14,28))
# ax[0].plot(Abnormal.Datetime, Abnormal.Current)
# ax[1].plot(Abnormal.Datetime, Abnormal.AX)
# ax[2].plot(Abnormal.Datetime, Abnormal.AY)
# ax[3].plot(Abnormal.Datetime, Abnormal.AZ)

In [19]:
condition = Abnormal.Datetime < datetime.datetime(2021, 6, 17, 22, 39)
Abnormal_filtered = Abnormal[condition]

In [20]:
# fig, ax = plt.subplots(4,1, figsize=(14,28))
# ax[0].plot(Abnormal_filtered.Datetime, Abnormal_filtered.Current)
# ax[1].plot(Abnormal_filtered.Datetime, Abnormal_filtered.AX)
# ax[2].plot(Abnormal_filtered.Datetime, Abnormal_filtered.AY)
# ax[3].plot(Abnormal_filtered.Datetime, Abnormal_filtered.AZ)

In [21]:
ds = pd.concat([Normal_filtered, Abnormal, Abnormal_fault])
ds.index.name = 'id'
# ds

In [22]:
ds.sort_values('id', inplace=True)
# ds

In [23]:
ds.shape

(1933337, 7)

In [24]:
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Flatten, Input

In [25]:
ds.drop('Datetime', axis=1, inplace=True)

In [26]:
del Data, Normal, Normal_filtered, Abnormal, Abnormal_fault, Abnormal_filtered

In [27]:
seq_len = 12**2
X = list()
y = list()
for i in range(seq_len, len(ds)-seq_len):
    # X.append(ds.iloc[i-seq_len:i][['Step', 'Current', 'AX', 'AY', 'AZ']].__array__())
    Step = ds.iloc[i-seq_len:i][['Step']].__array__().reshape((12, 12))
    Current = ds.iloc[i-seq_len:i][['Current']].__array__().reshape((12, 12))
    AX = ds.iloc[i-seq_len:i][['AX']].__array__().reshape((12, 12))
    AY = ds.iloc[i-seq_len:i][['AY']].__array__().reshape((12, 12))
    AZ = ds.iloc[i-seq_len:i][['AZ']].__array__().reshape((12, 12))
    X.append(np.dstack([Step, Current, AX, AY, AZ]))
    y.append(ds.iloc[i-1]['class'].__array__())

In [28]:
X, y = np.array(X), np.array(y)

In [29]:
X.shape

(1933049, 12, 12, 5)

In [30]:
y.shape

(1933049,)

In [31]:
from sklearn.model_selection import train_test_split

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:
X_train.shape

(1546439, 12, 12, 5)

In [34]:
y_train.shape

(1546439,)

In [35]:
X_test.shape

(386610, 12, 12, 5)

In [36]:
y_test.shape

(386610,)

In [37]:
del X, y

In [38]:
import os
if os.path.isfile("CNN.h5"):
    model = tf.keras.models.load_model("CNN.h5")
else:
    rf_input = Input(shape=(X_train.shape[1],X_train.shape[2], X_train.shape[3]))
    model = Conv2D(72, kernel_size=3, padding='same')(rf_input)
    model = Dropout(0.1)(model)
    model = Conv2D(16, kernel_size=3, padding='same')(model)
    model = Dropout(0.2)(model)
    model = Flatten()(model)
    model = Dense(16, activation='relu')(model)
    model = Dropout(0.2)(model)
    model = Dense(1, activation='relu')(model)
    model = tf.keras.Model(rf_input, model)

In [39]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 12, 12, 5)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 12, 12, 72)        3312      
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 72)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 12, 12, 16)        10384     
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 2304)              0         
_________________________________________________________________
dense (Dense)                (None, 16)               

In [40]:
model.compile(optimizer='adam', loss=tf.losses.binary_crossentropy, metrics=['accuracy'])

In [41]:
checkpoint_filepath = './tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True
)

In [42]:
model.fit(X_train,y_train, validation_data=(X_test, y_test), epochs=10, batch_size=1024,
          callbacks=[model_checkpoint_callback])

Epoch 1/10
1511/1511 [==============================] - 511s 338ms/step - loss: 7.5079 - accuracy: 0.5123 - val_loss: 5.1120 - val_accuracy: 0.6676

In [43]:
model.load_weights(checkpoint_filepath)
model.save("CNN.h5")

In [44]:
y_pred = model.predict(X_test)

In [45]:
y_pred.shape

(386610, 1)

In [46]:
y_pred_rs = y_pred.reshape(-1)
y_pred_rs.shape

(386610,)

In [47]:
y_test.shape

(386610,)

In [48]:
def F1_score(true_val, pred_val):
    TP = sum(true_val & pred_val)
    # TN = sum((~true_val - min(~true_val)) & (~pred_val - min(~pred_val)))
    FN = sum(true_val & (~pred_val - min(~pred_val)))
    FP = sum((~true_val - min(~true_val)) & pred_val)

    return TP / (TP + (FP + FN)/2)

In [49]:
result = F1_score(y_test.astype('int'), y_pred_rs.astype('int'))
result

0.6289862863499549